<a href="https://colab.research.google.com/github/nandhukumar86/CapstonePneumoniaDetection/blob/main/Vani/Capstoneyolomodel.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [12]:
TrainDir = "/content/drive/MyDrive/Content/DriveAIML/capstone/jpg_reshaped_train"
TestDir='/content/drive/MyDrive/Content/DriveAIML/capstone/jpg_reshaped_test'
basepath = "/content/drive/MyDrive/Content/DriveAIML/capstone/"

In [19]:
import os, cv2, glob,copy, gc
import pickle
import pandas as pd
import numpy as np
import datetime
from tqdm import tqdm_notebook
import seaborn as sns
import matplotlib.pyplot as plt
from matplotlib.patches import Rectangle
from sklearn import preprocessing
from sklearn.model_selection import train_test_split
import tensorflow as tf
from tensorflow.python.client import device_lib
from tensorflow import keras
from tensorflow.keras.applications import VGG16
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.preprocessing import image
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras import models, layers, Model, losses
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint, ReduceLROnPlateau
from tensorflow.keras.layers import Input, Concatenate, Dense, Dropout, Flatten, Activation, BatchNormalization

In [9]:
filename =  '/content/drive/MyDrive/Content/DriveAIML/capstone/RezisedLabel.pickle'
with open(filename, "rb") as input_file:
  y_train = pickle.load(input_file)

In [25]:
#trainimg = glob(TrainDir+'/*.jpg') #Getting all images in this folder
#testimg=glob(TestDir+"/*.jpg") #

ERROR: Could not find a version that satisfies the requirement Glob (from versions: none)
ERROR: No matching distribution found for Glob


In [10]:
y_train.head(5)


,patientId,x,y,width,height,Target,X,Y,Xo,Yo,xo,yo,widtho,heighto
0,0004cfab-14fd-4e49-80ba-63a80b6bddd6,NaN,NaN,NaN,NaN,0,1024,1024,256,256,NaN,NaN,NaN,NaN
1,00313ee0-9eaa-42f4-b0ab-c148ed3241cd,NaN,NaN,NaN,NaN,0,1024,1024,256,256,NaN,NaN,NaN,NaN
2,00322d4d-1c29-4943-afc9-b6754be640eb,NaN,NaN,NaN,NaN,0,1024,1024,256,256,NaN,NaN,NaN,NaN
3,003d8fa0-6bf1-40ed-b54c-ac657f8495c5,NaN,NaN,NaN,NaN,0,1024,1024,256,256,NaN,NaN,NaN,NaN
4,00436515-870c-4b36-a041-de91049b9ab4,264.0,152.0,213.0,379.0,1,1024,1024,256,256,66.0,38.0,54.0,95.0


In [11]:
y_train.size

423178

In [14]:
class_info_df = pd.read_csv(basepath+'stage_2_detailed_class_info.csv')

In [16]:
## Merging the two datasets
#In class detailed info dataset are given the detailed information about 
#the type of positive or negative class associated with a certain patient.
#In y_train labels dataset are given the patient ID and the window (x min, y min, width and height of the) 
#containing evidence of pneumonia
train_class_df = y_train.merge(class_info_df, left_on= ('patientId'), right_on=('patientId'), how='inner').drop_duplicates()
train_class_df.sample(5)

,patientId,x,y,width,height,Target,X,Y,Xo,Yo,xo,yo,widtho,heighto,class
12064,680297ea-1afd-40e5-b2cd-5a7d56a27024,NaN,NaN,NaN,NaN,0,1024,1024,256,256,NaN,NaN,NaN,NaN,No Lung Opacity / Not Normal
11654,6520e11d-e078-498a-9aa2-599eaab152ba,NaN,NaN,NaN,NaN,0,1024,1024,256,256,NaN,NaN,NaN,NaN,No Lung Opacity / Not Normal
8187,4b948dc1-39e8-4753-9ff8-11880f868b86,NaN,NaN,NaN,NaN,0,1024,1024,256,256,NaN,NaN,NaN,NaN,Normal
16196,8361600c-fe3d-4d7c-9f1b-1cb043713c0a,NaN,NaN,NaN,NaN,0,1024,1024,256,256,NaN,NaN,NaN,NaN,Normal
26059,bc1a6b13-b858-4900-bba3-03ac87e2257f,NaN,NaN,NaN,NaN,0,1024,1024,256,256,NaN,NaN,NaN,NaN,Normal


In [29]:
#reading image samples
sampNum = 2000 # Number of samples to import
X = [] # Empty to read training images

print("Before sampling\n")
print("Total training unique patient ids: ",train_class_df['patientId'].nunique())
print("Number of images in train set:", len(os.listdir(TrainDir)))
print("\n")
print("Total training labels {}" .format(train_class_df.shape))
#display(distTab(train_class_df, "Target")) # Dsitribution of target before sampling

print("\n\nAfter sampling\n")
Y = train_class_df.sample(n=sampNum, random_state=2) # Sampling
Y.reset_index(drop=True, inplace=True)
print("Total sampled training labels {}" .format(Y.shape))
#display(distTab(Y.sample(n=sampNum), "Target"))  # Dsitribution of target after sampling


for indx in Y.patientId:
    
    try:
        
        X.append(pydicom.read_file(trainXPth+indx+'.dcm'))
        
    except:
        
        continue
        

X = dict(zip([x.PatientID for x in X], [x.pixel_array for x in X])) # Dictionary for training images for patient id's
print("\n\nTotal sampled training images {}" .format(len(X)))

Y = Y[Y.patientId.isin(X.keys())]
print("Total training labels {}" .format(Y.shape))

Before sampling

Total training unique patient ids:  26684
Number of images in train set: 11740


Total training labels (30227, 15)


After sampling

Total sampled training labels (2000, 15)


Total sampled training images 0
Total training labels (0, 15)


In [34]:
# Shape of images
len(X)

0